<a href="https://colab.research.google.com/github/shenzhun/machine-learning-prep/blob/master/tensorflow/day13_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn import datasets

tf.Tensor([5.1 3.5 1.4 0.2], shape=(4,), dtype=float64) tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor([4.9 3.  1.4 0.2], shape=(4,), dtype=float64) tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor([4.7 3.2 1.3 0.2], shape=(4,), dtype=float64) tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor([4.6 3.1 1.5 0.2], shape=(4,), dtype=float64) tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor([5.  3.6 1.4 0.2], shape=(4,), dtype=float64) tf.Tensor(0, shape=(), dtype=int64)


In [ ]:
iris = datasets.load_iris()
ds1 = tf.data.Dataset.from_tensor_slices((iris["data"], iris["target"]))

for features, label in ds1.take(5):
  print(features, label)

tf.Tensor([5.1 3.5 1.4 0.2], shape=(4,), dtype=float64) tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor([4.9 3.  1.4 0.2], shape=(4,), dtype=float64) tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor([4.7 3.2 1.3 0.2], shape=(4,), dtype=float64) tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor([4.6 3.1 1.5 0.2], shape=(4,), dtype=float64) tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor([5.  3.6 1.4 0.2], shape=(4,), dtype=float64) tf.Tensor(0, shape=(), dtype=int64)


In [ ]:
import pandas as pd
dfiris = pd.DataFrame(iris["data"], columns = iris.feature_names)
ds2 = tf.data.Dataset.from_tensor_slices((dfiris.to_dict("list"),iris["target"]))

for features, label in ds2.take(3):
  print(features, label)

{'sepal length (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=5.1>, 'sepal width (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=3.5>, 'petal length (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=1.4>, 'petal width (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=0.2>} tf.Tensor(0, shape=(), dtype=int64)
{'sepal length (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=4.9>, 'sepal width (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=3.0>, 'petal length (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=1.4>, 'petal width (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=0.2>} tf.Tensor(0, shape=(), dtype=int64)
{'sepal length (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=4.7>, 'sepal width (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=3.2>, 'petal length (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=1.3>, 'petal width (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=0.2>} tf.Tensor(0, shape=(), dtype=int64)


In [ ]:
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_generator = ImageDataGenerator(rescale=1.0/255).flow_from_directory(
    "../data/cifar2/test/",
    target_size=(32, 32),
    batch_size=20,
    class_mode='binary'
)

classdict = image_generator.class_indices
print(classidct)

def generator():
  for features, label in image_generator:
    yield features, label

ds3 = tf.data.Dataset.from_generator(generator, output_types=(tf.float32, tf.int32))

FileNotFoundError: ignored

In [9]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.figure(figsize=(6,6)) 
for i,(img,label) in enumerate(ds3.unbatch().take(9)):
    ax=plt.subplot(3,3,i+1)
    ax.imshow(img.numpy())
    ax.set_title("label = %d"%label)
    ax.set_xticks([])
    ax.set_yticks([]) 
plt.show()

NameError: ignored

<Figure size 432x432 with 0 Axes>

In [10]:
# Constructing Data Pipeline through csv file
ds4 = tf.data.experimental.make_csv_dataset(
      file_pattern = ["../data/titanic/train.csv","../data/titanic/test.csv"],
      batch_size=3, 
      label_name="Survived",
      na_value="",
      num_epochs=1,
      ignore_errors=True)

for data,label in ds4.take(2):
    print(data,label)

ValueError: ignored

In [ ]:
ds5 = tf.data.TextLineDataset(
    filename = ["../data/titanic/train.csv","../data/titanic/test.csv"]
).skip(1)

for line in ds5.take(5):
  print(line)

In [ ]:
ds6 = tf.data.Dataset.list_files(
    ../data/cifar2/train/*/*.jpg"
)
for file in ds6.take(5):
  print(file)

In [ ]:
from matplotlib import pyplot as plt 
def load_image(img_path,size = (32,32)):
    label = 1 if tf.strings.regex_full_match(img_path,".*/automobile/.*") else 0
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img) # Note that we are using jpeg format
    img = tf.image.resize(img,size)
    return(img,label)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
for i,(img,label) in enumerate(ds6.map(load_image).take(2)):
    plt.figure(i)
    plt.imshow((img/255.0).numpy())
    plt.title("label = %d"%label)
    plt.xticks([])
    plt.yticks([])

In [ ]:
import os
import numpy as np

def create_tfrecords(inpath, outpath):
  writer = tf.io.TFRecordWriter(outpath)
  dirs = os.listdir(inpath)
  for index, name in enumerate(dirs):
    class_path = inpath + "/" + name + "/"
    for img_name in os.listdir(class_path):
      img_path = class_path + img_name
      img = tf.io.read_file(img_path)
      example = tf.train.Example(
          features = tf.train.Features(feature={
              'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
              'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img.numpy()]))
          })
      )
      writer.write(example.SerializeToString())
  writer.close()

create_tfrecords("../data/cifar2/test/","../data/cifar2_test.tfrecords/")

In [ ]:
from matplotlib import pyplot as plt 

def parse_example(proto):
    description ={ 'img_raw' : tf.io.FixedLenFeature([], tf.string),
                   'label': tf.io.FixedLenFeature([], tf.int64)} 
    example = tf.io.parse_single_example(proto, description)
    img = tf.image.decode_jpeg(example["img_raw"])   # Note that we are using jpeg format
    img = tf.image.resize(img, (32,32))
    label = example["label"]
    return(img,label)

ds7 = tf.data.TFRecordDataset("../data/cifar2_test.tfrecords").map(parse_example).shuffle(3000)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.figure(figsize=(6,6)) 
for i,(img,label) in enumerate(ds7.take(9)):
    ax=plt.subplot(3,3,i+1)
    ax.imshow((img/255.0).numpy())
    ax.set_title("label = %d"%label)
    ax.set_xticks([])
    ax.set_yticks([]) 
plt.show()

Applying Data Conversion

In [12]:
ds = tf.data.Dataset.from_tensor_slices(["hello world", "hello China", "hello Beijing"])
ds_map = ds.map(lambda x: tf.strings.split(x, " "))
for x in ds_map:
  print(x)

tf.Tensor([b'hello' b'world'], shape=(2,), dtype=string)
tf.Tensor([b'hello' b'China'], shape=(2,), dtype=string)
tf.Tensor([b'hello' b'Beijing'], shape=(2,), dtype=string)


In [14]:
ds = tf.data.Dataset.from_tensor_slices(["hello world", "hello China", "hello Beijing"])
ds_flatmap = ds.flat_map(lambda x: tf.data.Dataset.from_tensor_slices(tf.strings.split(x, " ")))
for x in ds_flatmap:
  print(x)

tf.Tensor(b'hello', shape=(), dtype=string)
tf.Tensor(b'world', shape=(), dtype=string)
tf.Tensor(b'hello', shape=(), dtype=string)
tf.Tensor(b'China', shape=(), dtype=string)
tf.Tensor(b'hello', shape=(), dtype=string)
tf.Tensor(b'Beijing', shape=(), dtype=string)


In [16]:
ds = tf.data.Dataset.from_tensor_slices(["hello world","hello China","hello Beijing"])
ds_interleave = ds.interleave(lambda x: tf.data.Dataset.from_tensor_slices(tf.strings.split(x," ")))
for x in ds_interleave:
  print(x)

tf.Tensor(b'hello', shape=(), dtype=string)
tf.Tensor(b'hello', shape=(), dtype=string)
tf.Tensor(b'world', shape=(), dtype=string)
tf.Tensor(b'China', shape=(), dtype=string)
tf.Tensor(b'hello', shape=(), dtype=string)
tf.Tensor(b'Beijing', shape=(), dtype=string)


In [17]:
ds = tf.data.Dataset.from_tensor_slices(["hello world","hello China","hello Beijing"])
ds_filter = ds.filter(lambda x: tf.strings.regex_full_match(x, ".*[a|B].*"))
for x in ds_filter:
  print(x)

tf.Tensor(b'hello China', shape=(), dtype=string)
tf.Tensor(b'hello Beijing', shape=(), dtype=string)


In [20]:
ds1 = tf.data.Dataset.range(0, 3)
ds2 = tf.data.Dataset.range(3, 6)
ds3 = tf.data.Dataset.range(6, 9)
ds_zip = tf.data.Dataset.zip((ds1, ds2, ds3))
for x, y, z in ds_zip:
  print(x.numpy(), y.numpy(), z.numpy())

0 3 6
1 4 7
2 5 8


In [21]:
ds1 = tf.data.Dataset.range(0, 3)
ds2 = tf.data.Dataset.range(3, 6)
ds_concat = tf.data.Dataset.concatenate(ds1, ds2)
for x in ds_concat:
  print(x)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)


In [22]:
ds = tf.data.Dataset.range(12)
ds_batch = ds.batch(4)
for x in ds_batch:
  print(x)

tf.Tensor([0 1 2 3], shape=(4,), dtype=int64)
tf.Tensor([4 5 6 7], shape=(4,), dtype=int64)
tf.Tensor([ 8  9 10 11], shape=(4,), dtype=int64)


In [23]:
elements = [[1, 2], [3, 4, 5], [6, 7], [8]]
ds = tf.data.Dataset.from_generator(lambda: iter(elements), tf.int32)

ds_padded_batch = ds.padded_batch(2, padded_shapes=[4,])
for x in ds_padded_batch:
  print(x)

tf.Tensor(
[[1 2 0 0]
 [3 4 5 0]], shape=(2, 4), dtype=int32)
tf.Tensor(
[[6 7 0 0]
 [8 0 0 0]], shape=(2, 4), dtype=int32)


In [24]:
ds = tf.data.Dataset.range(12)
ds_window = ds.window(3, shift=1).flat_map(lambda x: x.batch(3, drop_remainder=True))
for x in ds_window:
  print(x)

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
tf.Tensor([1 2 3], shape=(3,), dtype=int64)
tf.Tensor([2 3 4], shape=(3,), dtype=int64)
tf.Tensor([3 4 5], shape=(3,), dtype=int64)
tf.Tensor([4 5 6], shape=(3,), dtype=int64)
tf.Tensor([5 6 7], shape=(3,), dtype=int64)
tf.Tensor([6 7 8], shape=(3,), dtype=int64)
tf.Tensor([7 8 9], shape=(3,), dtype=int64)
tf.Tensor([ 8  9 10], shape=(3,), dtype=int64)
tf.Tensor([ 9 10 11], shape=(3,), dtype=int64)


In [26]:
#shuffle: shuffling the order of the data.

ds = tf.data.Dataset.range(12)
ds_shuffle = ds.shuffle(buffer_size=5)
for x in ds_shuffle:
  print(x)

tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(10, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(11, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [27]:
# repeat: repeat the data centain times

ds = tf.data.Dataset.range(3)
ds_repeat = ds.repeat(3)
for x in ds_repeat:
  print(x)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)


In [28]:
#shard: sampling the elements starting from a certain position with fixed distance.

ds = tf.data.Dataset.range(12)
ds_shard = ds.shard(3, index = 1)
for x in ds_shard:
  print(x)

tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(10, shape=(), dtype=int64)


In [29]:
#take: sampling the first few elements from a certain position.

ds = tf.data.Dataset.range(12)
ds_take = ds.take(3)

list(ds_take.as_numpy_iterator())

[0, 1, 2]

In [31]:
import tensorflow as tf

@tf.function
def printbar():
  ts = tf.timestamp()
  today_ts = ts % (24 * 60 * 60)

  hour = tf.cast(today_ts//3600+8,tf.int32)%tf.constant(24)
  minite = tf.cast((today_ts%3600)//60,tf.int32)
  second = tf.cast(tf.floor(today_ts%60),tf.int32)

  def timeformat(m):
    if tf.strings.length(tf.strings.format("{}", m)) == 1:
      return tf.strings.format("0{}", m)
    else:
      return tf.strings.format("{}", m)
  
  timestring = tf.strings.join([timeformat(hour),timeformat(minite),
                               timeformat(second)], separator=":")
  tf.print("=========" * 8, end = "")
  tf.print(timestring)

In [33]:
import time

def generator():
  for i in range(10):
    time.sleep(2)
    yield i

ds = tf.data.Dataset.from_generator(generator, output_types=tf.int32)

def train_step():
  time.sleep(1)

In [34]:
# Estimated time of training: 10*2+10*1 = 30s
printbar()
tf.print(tf.constant("start training..."))
for x in ds:
    train_step()  
printbar()
tf.print(tf.constant("end training..."))

========================================================================14:12:56
start training...
========================================================================14:13:26
end training...


In [35]:
# Use method prefetch to parallel the processes of data preparation and parameter iteration.

# Estimated time of training:  max(10*2,10*1) = 20s
printbar()
tf.print(tf.constant("start training with prefetch..."))

# tf.data.experimental.AUTOTUNE allows auto-selection of parameters
for x in ds.prefetch(buffer_size = tf.data.experimental.AUTOTUNE):
    train_step()  
    
printbar()
tf.print(tf.constant("end training..."))

========================================================================14:13:27
start training with prefetch...
========================================================================14:13:48
end training...


In [36]:
ds_files = tf.data.Dataset.list_files("../data/titanic/*.csv")
ds = ds_files.flat_map(lambda x:tf.data.TextLineDataset(x).skip(1))
for line in ds.take(4):
    print(line)

InvalidArgumentError: ignored

In [37]:
ds_files = tf.data.Dataset.list_files("../data/titanic/*.csv")
ds = ds_files.interleave(lambda x:tf.data.TextLineDataset(x).skip(1))
for line in ds.take(8):
    print(line)

InvalidArgumentError: ignored

In [ ]:
ds = tf.data.Dataset.list_files("../data/cifar2/train/*/*.jpg")
def load_image(img_path,size = (32,32)):
    label = 1 if tf.strings.regex_full_match(img_path,".*/automobile/.*") else 0
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img) #Note: jpeg format here
    img = tf.image.resize(img,size)
    return(img,label)

In [ ]:
# Conversion with single process
printbar()
tf.print(tf.constant("start transformation..."))

ds_map = ds.map(load_image)
for _ in ds_map:
    pass

printbar()
tf.print(tf.constant("end transformation..."))

In [ ]:
# Conversion with multi-process
printbar()
tf.print(tf.constant("start parallel transformation..."))

ds_map_parallel = ds.map(load_image,num_parallel_calls = tf.data.experimental.AUTOTUNE)
for _ in ds_map_parallel:
    pass

printbar()
tf.print(tf.constant("end parallel transformation..."))


In [38]:
import time

def generator():
  for i in range(5):
    time.sleep(2)
    yield i

ds = tf.data.Dataset.from_generator(generator, output_types= (tf.int32))

def train_step():
  pass

printbar()
tf.print(tf.constant("start training..."))

for epoch in tf.range(3):
  for x in ds:
    train_step()
  
  printbar()
  tf.print("epoch =", epoch, " ended")
printbar()
tf.print(tf.constant("end training.."))

========================================================================14:25:55
start training...
========================================================================14:26:05
epoch = 0  ended
========================================================================14:26:15
epoch = 1  ended
========================================================================14:26:25
epoch = 2  ended
========================================================================14:26:25
end training..


In [41]:
import time

def generator():
  for i in range(5):
    time.sleep(2)
    yield i

ds = tf.data.Dataset.from_generator(generator, output_types = (tf.int32)).cache()

def train_step():
  time.sleep(0)

printbar()
tf.print(tf.constant("start training..."))
for epoch in tf.range(3):
  for x in ds:
    train_step()
  
  printbar()
  tf.print("epoch =", epoch, " ended")
printbar()
tf.print(tf.constant("end training..."))

========================================================================14:28:59
start training...
========================================================================14:29:09
epoch = 0  ended
========================================================================14:29:09
epoch = 1  ended
========================================================================14:29:09
epoch = 2  ended
========================================================================14:29:09
end training...


In [42]:
ds = tf.data.Dataset.range(100000)
ds_map_batch = ds.map(lambda x: x**2).batch(20)

printbar()
tf.print(tf.constant("start scalar transoformation..."))
for x in ds_map_batch:
  pass

printbar()
tf.print(tf.constant("end scalar transformation..."))

========================================================================14:31:05
start scalar transoformation...
========================================================================14:31:08
end scalar transformation...


In [44]:
ds = tf.data.Dataset.range(100000)
ds_batch_map = ds.batch(20).map(lambda x: x**2)

printbar()
tf.print(tf.constant("start vector transformation..."))
for x in ds_batch_map:
  pass
printbar()
tf.print(tf.constant("end vecotr transformation..."))


========================================================================14:32:36
start vector transformation...
========================================================================14:32:37
end vecotr transformation...
